COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name: Santiago Sanchez
#### Date: jan 19 2026

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [26]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures
from dataclasses import dataclass, field

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [29]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

# Sample usage:  get the list of NEOs for a given week, then lookup the latest NEO record in that list.
week_start = '2023-01-15'
neos = get_neos(week_start)
print(f'{len(neos)} Near Earth Objects found for week of {week_start}')
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

KeyError: 'near_earth_objects'

## Exercise 1: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a *single* close approach record.
* State variables are
    * orbiting body (`str`)
    * approach date (`datetime` object!)
    * miss distance (`float` in km, document it!)
    * relative velocity (`float` in km/hr, ditto)

* Operations must include:
    * `__init__(self, ...)` method to initialize a new object with specific data values
    * `__str__(self)` method to return a nicely formatted string representation of the object.

Write a little code to test your new class.

In [18]:
# Ex. 1 your code here
@dataclass
class CloseApproach:
  '''Class to represent a SINGLE close approach record '''

  orbiting_body: str
  apporach_date: datetime #Get object datetime
  miss_distance: float
  velocity:float

  def __str__(self):
    '''Returns string representation of the object '''

    return f'''{self.orbiting_body} was closed on {self.apporach_date}. The miss distance was {self.miss_distance} km and the relative velocity was {self.velocity} km/hr'''

In [19]:
# Testing unit
date = datetime.datetime.now()
neo1 = CloseApproach(orbiting_body="Asteroid", apporach_date=date, miss_distance=888999.12322, velocity= 12300)
str(neo1)

'Asteroid was closed on 2026-01-20 04:39:15.073806. The miss distance was 888999.12322 km and the relative velocity was 12300 km/hr'

## Exercise 2: Factory function: get_close_approach

We want to be able to construct CloseApproach objects easily from a data record returned from the NEO API.  

Write an "object factory" function...   

    def get_close_approach(record):
        ...

This function provides an easy way to create a `CloseApproach` instance.  It takes a dictionary for a single `close_approach_data` record, constructs and returns a `CloseApproach` object representing that same record.
This kind of function is called a “Factory” because it handles the details of constructing an object from raw materials.

Remember to convert each element from the data dictionary to the correct type (e.g., parse the date/time string into a `datetime` object).
Add little code to test your new factory function.

pseudocode:
takes dicitonary, from the dicitonary, each key may be something we need to create the objecet
Therefore, filter, take, and then repeat with each data needed, then return object
Data needed:

- orbiting body

- approach date, turned to date object

- miss distance

- relative_velocity in kilometers

In [24]:
# Ex. 2 your code here

def get_close_approach(my_dictionary: dict) -> CloseApproach:
  '''Takes a dicitonary from data and Returns a CloaseApproach Object '''

  #Take individual data
  orbiting = my_dictionary["orbiting_body"] #Get dictionary from list of neos
  approach_date = datetime.date.fromisoformat(my_dictionary["close_approach_date"])
  miss_distance = float(my_dictionary['miss_distance']['kilometers'])
  relative_velocity = float(my_dictionary['relative_velocity']['kilometers_per_hour'])

  return CloseApproach(orbiting_body=orbiting, apporach_date=approach_date, miss_distance=miss_distance, velocity=relative_velocity)


new_object = get_close_approach(neo["close_approach_data"][0])
print(new_object)
assert type(new_object.apporach_date) == datetime.date
assert type(new_object.miss_distance) == float


KeyError: 'close_approach_data'

## Exercise 3:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables representing a single NEO.  Your Asteroid class should define at least 4 "state variables:”
* id  (`int`)
* name (`str`)
* estimated_diameter (`float` in m)
* is_potentially_hazardous (`bool`)
* close_approaches (`list` of CloseApproach objects, default to empty list)

Operations must include:
* `__init__(self, ...)` method to initialize a new object with specific data values
* `__str__(self)` method to return a nicely formatted string representation of the object.

Write a little code to test you new class (just leave close_approaches as an empty list for now).

Pseudocode
asteroid class will take
  - id type int
  - name str
  - estimated_diameter type float in meters
  - is_potenially_hazardous bool
  - close_apporches llist of closeApproach objects by defaul is []

  methods init and str are expectred

In [9]:
# Ex. 3 your code here
@dataclass
class Asteroid:

  id: int
  name: str
  estimated_diameter: float #in meters
  is_potentially_hazardous: bool #either True or False
  close_approaches: list = field(default_factory=list)

  def __str__(self) -> str:
    '''Returns description of class Asteroid '''

    return f"{self.name} with id: {self.id}, with an estimated diameter of {self.estimated_diameter}, therefore its level of hazardous is {self.is_potentially_hazardous}"

new_asteroid = Asteroid(id=1223, name="Big Asteroid", estimated_diameter=1212.78, is_potentially_hazardous=False)

str(new_asteroid)

'Big Asteroid with id: 1223, with an estimated diameter of 1212.78, therefore its level of hazardous is False'

## Exercise 4: Asteroid factory

Write a function that returns an Asteroid object just from the id for a single NEO.

    def asteroid_from_neo(neo_id):
        ...

This factory function takes the `id` for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.  *Hint*: I provided the code fetch a NEO from its `id` above.

Every `Asteroid` should have a list of “close approaches”.
*Hint*: use the `get_close_approach` factory you defined above to construct the required list of CloseApproach objects.

Now add a new method to `Asteroid` class to return the `CloseApproach` object from the asteroid representing its nearest to **Earth**:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

code a funciton that takes an id for a NEO recorder from the API
pass that ID to the get_neo function already written

you should create an Asteroid object for thE given id, use the Asteroid class
  you will need:

  ID, name, Estimated diameter, hazardous and close approaces:


  the first 4 are an specific key inside the neo dictionary,

        
  for the last one, use a list comprehension to get all the close approaches someting, it will return a list of CloseApproach object, and those objects will be in the list


  use tghe get_close_approach, since you will be given th edictionary on index [0] of first potion of noe list

In [15]:
# Ex. 4 your test code for nearest_miss here
def asteroid_from_neo(neo_id):

  neo = get_neo()

SyntaxError: incomplete input (ipython-input-694892726.py, line 2)

## Challenge - Take your skills to the next level...
### Exercise 5: develop a useful analysis / data product

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  nearest_miss
    * apply Python’s min function to identify the Asteroid with the nearest_miss

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, `estimated_mass`, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the `CloseApproach` class, `impact_force`,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Ex. 5 (challenge) your code here